In [5]:
import requests
import json
import pprint
import pandas as pd
#!pip install xmltodict
import xmltodict
from xml.etree import ElementTree
#!pip install pymed
from pymed import PubMed
#%load_ext google.colab.data_table
#!pip install plotly
import plotly
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime

In [6]:
#https://stackoverflow.com/questions/57053378/query-pubmed-with-python-how-to-get-all-article-details-from-query-to-pandas-d
pubmed = PubMed(tool="PubMedSearcher", email="martina.fonseca@nhsx.nhs.uk")
results = pubmed.query('nhsx[affiliation]', max_results=500) # number might need to be updated in future, for now low
articleList = []
articleInfo = []

for article in results:
# Print the type of object we've found (can be either PubMedBookArticle or PubMedArticle).
    articleDict = article.toDict() # convert to dictionary
    articleList.append(articleDict)

# Generate list of dict records which will hold all article details that could be fetch from PUBMED API
for article in articleList:
#Sometimes article['pubmed_id'] contains list separated with comma - take first pubmedId in that list - thats article pubmedId
    pubmedId = article['pubmed_id'].partition('\n')[0] # keep only pubmed id
    # Append article info to dictionary #
    articleInfo.append({u'pubmed_id':pubmedId,
                       u'title':article['title'],
                       u'keywords':article['keywords'],
                       u'journal':article['journal'],
                       u'abstract':article['abstract'],
                       u'conclusions':article['conclusions'],
                       u'methods':article['methods'],
                       u'results': article['results'],
                       u'copyrights':article['copyrights'],
                       u'doi':article['doi'],
                       u'publication_date':article['publication_date'], 
                       u'authors':article['authors']})

# Convert list of dictionaries to pandas DataFrame
articlesPD = pd.DataFrame.from_dict(articleInfo)
#export_csv = df.to_csv (r'C:\Users\YourUsernam\Desktop\export_dataframe.csv', index = None, header=True) 

#Print first 10 rows of dataframe
articlesPD.head(10)
#print(articlesPD['title'])
#print(articlesPD['keywords'])
#print(articleList)

,pubmed_id,title,keywords,journal,abstract,conclusions,methods,results,copyrights,doi,publication_date,authors
0,33094226,Digital health - a trainee's perspective.,"[QI, Trainee, digital, training]",Future healthcare journal,"As we shift towards using digital systems, and...",None,None,None,© Royal College of Physicians 2020. All rights...,10.7861/fhj.dig-2020-trai,2020-10-24,"[{'lastname': 'Maguire', 'firstname': 'James',..."
1,32702587,The ethics of AI in health care: A mapping rev...,"[Artificial intelligence, Ethics, Health polic...",Social science & medicine (1982),This article presents a mapping review of the ...,None,None,None,Copyright © 2020 Elsevier Ltd. All rights rese...,10.1016/j.socscimed.2020.113172,2020-07-24,"[{'lastname': 'Morley', 'firstname': 'Jessica'..."
2,32672131,Bringing NHS data analysis into the 21st century.,[],Journal of the Royal Society of Medicine,None,None,None,None,None,10.1177/0141076820930666,2020-07-17,"[{'lastname': 'Goldacre', 'firstname': 'Ben', ..."
3,32616598,Using imaging to combat a pandemic: rationale ...,[],The European respiratory journal,None,None,None,None,None,10.1183/13993003.01809-2020,2020-07-04,"[{'lastname': 'Jacob', 'firstname': 'Joseph', ..."
4,32010451,Supporting early clinical careers in digital h...,[],Digital health,None,None,None,None,None,10.1177/2055207619899798,2020-02-06,"[{'lastname': 'Robbins', 'firstname': 'Tim', '..."


In [13]:
articlesPD_sel = articlesPD[["title","journal","publication_date","doi"]]

In [ ]:
df.rename(columns={"title": "Publication title", "B": "c"})

In [15]:
articlesPD_sel.head()

,title,journal,publication_date,doi
0,Digital health - a trainee's perspective.,Future healthcare journal,2020-10-24,10.7861/fhj.dig-2020-trai
1,The ethics of AI in health care: A mapping rev...,Social science & medicine (1982),2020-07-24,10.1016/j.socscimed.2020.113172
2,Bringing NHS data analysis into the 21st century.,Journal of the Royal Society of Medicine,2020-07-17,10.1177/0141076820930666
3,Using imaging to combat a pandemic: rationale ...,The European respiratory journal,2020-07-04,10.1183/13993003.01809-2020
4,Supporting early clinical careers in digital h...,Digital health,2020-02-06,10.1177/2055207619899798


In [16]:
# Create output table
fig = go.Figure()
fig.add_trace(
    go.Table(
        header=dict(
            values=["<b>" + c + "<b>" for c in articlesPD_sel.columns],
            fill_color="rgba(240, 244, 245, 1)",
            align="left",
        ),
        cells=dict(
            values=articlesPD_sel.T.values.tolist(),
            fill_color="rgba(240, 244, 245, 1)",
            align="left",
        ),
    )
)

# Asthetics of the table
fig.update_layout(
    {
        "plot_bgcolor": "rgba(240, 244, 245, 1)",
        "paper_bgcolor": "rgba(240, 244, 245, 1)",
    },
    autosize=True,
    height=500,
    width=1000
)


# Write out to file (.html)
config = {"displayModeBar": False, "displaylogo": False}
plotly_table = plotly.offline.plot(
    fig, include_plotlyjs=False, output_type="div", config=config
)
with open("_includes/plotly_table_nhsx.html", "w") as file:
    file.write(plotly_table)

In [11]:
# Grab timestamp
data_updated = datetime.now().strftime("%d/%m/%Y")

# Write out to file (.html)
html_str = (
    '<p><svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 16 16" width="16" height="16"><path fill-rule="evenodd" d="M1.5 8a6.5 6.5 0 1113 0 6.5 6.5 0 01-13 0zM8 0a8 8 0 100 16A8 8 0 008 0zm.5 4.75a.75.75 0 00-1.5 0v3.5a.75.75 0 00.471.696l2.5 1a.75.75 0 00.557-1.392L8.5 7.742V4.75z"></path></svg> Latest Data: '
    + data_updated
    + "</p>"
)
with open("_includes/update.html", "w") as file:
    file.write(html_str)